
<img align="left" src = images/linea.png width=150 style="padding: 20px"> <br> 
<img align="left" src = https://jupyter.org/assets/homepage/hublogo.svg width=200 style="padding: 20px"> <br> 

# JupyterHub Tutorial

<br>
<br>


## Notebook 2 - Acesso a dados



Contato: Julia Gschwend (julia@linea.org.br)

Última verificação: 06/08/2024
***


Bem vindo(a) ao LIneA JupyterHub! 

O LIneA JupyterHub oferece acesso a dados públicos disponíveis online e a dados privados de levantamentos fotométricos cuja participação de cientistas brasileiros com _data rights_ é apoiada pelo LIneA. Neste notebook vamos exemplificar o acesso aos dados públicos do levantamento _Dark Energy Survey_ (DES). Caso precise de ajuda, entre em contato pelo e-mail: [helpdesk@linea.org.br](mailto:helpdesk@linea.org.br)


## 1. Sobre os dados

<img align="left" src=https://www.darkenergysurvey.org/wp-content/uploads/2016/01/des-logo-rev-lg.png width=400 style="background-color:black; padding: 20px">


<img align="left" src=https://www.darkenergysurvey.org/wp-content/uploads/2016/05/12-0333-22D.jpg width=400 style="background-color:black; padding: 20px; margin-right: 2em"> <br>




O [Dark Energy Survey (DES)](https://www.darkenergysurvey.org/) é um levantamento fotométrico em 5 bandas do ótico ao infravermelho (_grizY_) que tem como principal objetivo a determinação da equação de estado da energia escura. O DES observou ~700 milhões de objetos detectados em ~5000 graus quadrados no hemisfério sul durante 6 anos. Os artigos com os principais resultados da análise dos dados dos três primeiros anos de observação estão disponíveis [nesta página](https://www.darkenergysurvey.org/des-year-3-cosmology-results-papers/).  
<img align="center" src=https://www.darkenergysurvey.org/wp-content/uploads/2021/06/dr2_footprint.png  width=500 style="padding: 20px"> <br> 
Figura: Footprint Data Release 2 (fonte: [www.darkenergysurvey.org](https://www.darkenergysurvey.org/wp-content/uploads/2021/06/dr2_footprint.png)).  

O [segundo _data release_ (DR2)](https://des.ncsa.illinois.edu/releases/dr2), já contendo os dados dos seis anos de observação, está disponível para público e pode ser acessado pelo [LIneA Science Server](https://desportal2.cosmology.illinois.edu/sky/) ou aqui pelo JupyterHub, como veremos a seguir.

## 2. Acesso ao banco de dados 

Dentro da plataforma LIneA JupyterHub, o acesso ao banco de dados é feito através da biblioteca [dblinea](https://github.com/linea-it/dblinea). Confira a documentação completa da biblioteca [neste link](https://dblinea.readthedocs.io/en/latest/index.html).


Instalação da biblioteca `dblinea`: 

In [ ]:
! pip install dblinea

Nos exemplos abaixo, também vamos usar a biblioteca [Astropy](https://docs.astropy.org/en/stable/). As demais bibliotecas utilizadas já estão disponíveis na instalação original do JupyterHub. 

Instalação da biblioteca `astropy`: 

In [ ]:
! pip install astropy

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord

from dblinea import DBBase

%reload_ext autoreload
%autoreload 2

**Leitura dos dados**


A classe `DBBase` faz a conexão com o banco de dados e oferece algumas funcionalidades como veremos a seguir. Nos exemplos abaixo, vamos utilizar o objeto `db` para acessar os dados e metadados da tabela "**main**" do segundo _release_ (**DR2**) do levantamento **DES**. 

In [ ]:
db = DBBase()

In [ ]:
schema = "des_dr2"  
tablename = "main"

O método `get_table_columns` retorna a lista de colunas disponíveis na tabela: 

In [ ]:
db.get_table_columns(tablename, schema=schema)

O método `describe_table` retorna o tipo de dado em cada coluna: 

In [ ]:
db.describe_table(tablename, schema=schema)

A função `fetchall(query)` faz a consulta no banco de dados e retorna uma **lista de tuplas** com os dados referentes à _query_ fornecida no argumento. Por exemplo, vamos consultar o identificador único (`coadd_object_id`) e as coordenadas equatoriais dos objetos nas 10 primeiras linhas da tabela.  

In [ ]:
query = f"SELECT coadd_object_id, ra, dec FROM {schema}.{tablename} limit 10"
lista_10_objetos = db.fetchall(query)
lista_10_objetos

A função `fetchall_dict(query)` faz a consulta no banco de dados e retorna uma **lista de dicionários** com os dados referentes à _query_ fornecida no argumento. Vamos repetir a consulta do exemplo anterior, ou seja, vamos utilizar o mesmo _SQL statement_ atribuído à variável `query`.    

In [ ]:
dict_10_objetos = db.fetchall_dict(query)
dict_10_objetos

A função `fetchall_df(query)` faz a consulta no banco de dados e retorna um objeto do tipo [**pandas.DataFrame**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) com os dados referentes à _query_ fornecida no argumento. Vamos repetir a consulta dos exemplo anteriores:

In [ ]:
dataframe_10_objetos = db.fetchall_df(query)
dataframe_10_objetos

### Exemplos 

Para ilustrar a visualização de uma pequena amostra de dados, vamos construir o diagrama cor-magnitude com as estrelas da [galáxia anã de Sculptor](https://en.wikipedia.org/wiki/Sculptor_Dwarf_Galaxy). 

|Coordenadas Equatoriais| |
|:-- |--- | 
|Ascensão Reta| 01h 00m 09.3s |
|Declinação| −33° 42' 33" |




A tabela abaixo traz os significados das colunas que vamos utilizar para fazer a consulta no banco de dados. A lista completa de colunas disponíveis na tabela **DES DR2** está disponível [nesta página](https://des.ncsa.illinois.edu/releases/dr2/dr2-products/dr2-schema). 


|Coluna | Significado |
|---|---|
|COADD_OBJECT_ID | Unique identifier for the coadded objects|
|RA | Right ascension, with quantized precision for indexing (ALPHAWIN_J2000 has full precision but not indexed) [degrees]|
|DEC | Declination, with quantized precision for indexing (DELTAWIN_J2000 has full precision but not indexed) [degrees] |
|EXTENDED_CLASS_COADD |0: high confidence stars; 1: candidate stars; 2: mostly galaxies; 3: high confidence galaxies; -9: No data; Using Sextractor photometry |
|FLAGS_{G,R,I,Z,Y}| Additive flag describing cautionary advice about source extraction process. Use less than 4 for well behaved objects |
|MAG_AUTO_{G,R,I,Z,Y} | Magnitude estimation, for an elliptical model based on the Kron radius [mag] |
|MAG_AUTO_{G,R,I,Z,Y}_DERED | Dereddened magnitude estimation (using SFD98), for an elliptical model based on the Kron radius [mag]|




Para fazer uma busca pelas coordenadas no banco, precisamos converter as unidades para graus. Para isto, vamos usar a classe `SkyCoord` do módulo `astropy.coordinates` (veja detalhes na [documentação do Astropy](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html)):

In [ ]:
c = SkyCoord('01h00m09.3s', '−33d42m33s', frame='icrs')
c  

In [ ]:
print(f"R.A.: {c.ra.deg:.1f} degrees")
print(f"Dec.: {c.dec.deg:.1f} degrees")

Os dois exemplos abaixos mostram a consulta das magnitudes nas bandas _g, r, i_ corrigidas do avermelhamento (sufixo _dered_) e seus respectivos erros, para uma seleção de objetos classificados como estrelas (`extended_class_coadd < 2`, vide tabela acima).

#### **Exemplo 1: seleção de uma região "retangular"**

Para selecionar qualquer amostra com base nas coordenadas de posição, recomenda-se utilizar as funções da biblioteca [Q3C](https://github.com/segasai/q3c) ([Koposov, S., & Bartunov, O. 2006](http://adsabs.harvard.edu/abs/2006ASPC..351..735K)) para tirar vantagem da indexação das colunas. Uma busca por uma região no céu definida por faixas de coordenadas resultaria em uma varredura em todos os ~700 milhões de objetos, o que pode levar um tempo considerável!   


Para consultar objetos dentro de uma região contida em um polígono, basta informar os vértices do polígono pra a função `q3c_poly_query()`. A documentação das funções está disponível no [repositório do Q3C](https://github.com/segasai/q3c).  

Vamos selecionar as magnitudes de uma amostra de estrelas em uma região "quadrada" (na esfera celeste) de lado igual a 1 grau, ou seja, com uma margem de 0.5 graus em torno da posição do nosso alvo. 

**Vértices (ra, dec)**: (14.5, -34.2), (15.5, -34.2), (15.5, -33.2), (14.5, -33.2)

A query ficaria assim (o uso de letras maiúsculas é opcional):  

```sql 
SELECT coadd_object_id, ra ,dec, flags_g, mag_auto_g_dered, mag_auto_r_dered, 
        mag_auto_i_dered, magerr_auto_g, magerr_auto_r, magerr_auto_i 
FROM DES_DR2.MAIN 
WHERE q3c_poly_query(ra, dec, ARRAY[14.5, -34.2, 15.5, -34.2, 15.5, -33.2, 14.5, -33.2])
AND extended_class_coadd < 2 
````

In [ ]:
query_1 = f"SELECT coadd_object_id, ra ,dec, flags_g, mag_auto_g_dered, mag_auto_r_dered, mag_auto_i_dered, magerr_auto_g, magerr_auto_r, magerr_auto_i FROM {schema}.{tablename} WHERE q3c_poly_query(ra, dec, ARRAY[14.5, -34.2, 15.5, -34.2, 15.5, -33.2, 14.5, -33.2]) AND extended_class_coadd < 2 "        
query_1 

In [ ]:
%%time
dados_exemplo_1 = db.fetchall_df(query_1)

In [ ]:
dados_exemplo_1.head()

#### **Exemplo 2: seleção de uma região  circular com Q3C**  

Para selecionar uma região circular, basta informar as coordenadas do centro da seleção e um raio (em graus) nos argumentos da função `q3c_radial_query()`. Para um diâmetro de 1 grau, a query ficaria assim:
    
```sql
SELECT coadd_object_id, ra ,dec, flags_g, mag_auto_g_dered, mag_auto_r_dered, 
        mag_auto_i_dered, magerr_auto_g, magerr_auto_r, magerr_auto_i, 
FROM DES_DR2.MAIN 
WHERE q3c_radial_query(ra, dec, 15.0, -33.7, 0.5)
AND extended_class_coadd < 2 
````

In [ ]:
query_2 = f"SELECT coadd_object_id, ra ,dec, flags_g, mag_auto_g_dered, mag_auto_r_dered, mag_auto_i_dered, magerr_auto_g, magerr_auto_r, magerr_auto_i FROM {schema}.{tablename} WHERE q3c_radial_query(ra, dec, 15.0, -33.7, 0.5) AND extended_class_coadd < 2 "
query_2    

In [ ]:
%%time
dados_exemplo_2 = db.fetchall_df(query_2)

In [ ]:
dados_exemplo_2.head()

### Gráficos 

A seguir, veremos exemplos de gráficos estáticos básicos com a biblioteca Matplotlib. 

Documentação da biblioteca: [matplotlib.org](https://matplotlib.org/)

Dicas: [Cheatsheets for Matplotlib users](https://github.com/matplotlib/cheatsheets#cheatsheets)

#### Distribuição espacial

In [ ]:
%%time
plt.figure(figsize=[10,5], dpi=300)
plt.suptitle("Stars from DES DR2", fontsize=14)
plt.subplot(1,2,1)
plt.plot(dados_exemplo_1.ra, dados_exemplo_1.dec, 'k.', alpha=0.1)
plt.xlabel("R.A. (deg)", fontsize=14)
plt.ylabel("Dec. (deg)", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.subplot(1,2,2)
plt.plot(dados_exemplo_2.ra, dados_exemplo_2.dec, 'k.', alpha=0.1)
plt.xlabel("R.A. (deg)", fontsize=14)
plt.ylabel("Dec. (deg)", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

#### Mapa de densidade

In [ ]:
%%time
plt.figure(figsize=[12,5], dpi=300)
plt.suptitle("Stars from DES DR2", fontsize=14)
plt.subplot(1,2,1)
plt.hist2d(dados_exemplo_1.ra, dados_exemplo_1.dec, bins=50)
plt.xlabel("R.A. (deg)", fontsize=14)
plt.ylabel("Dec. (deg)", fontsize=14)
plt.colorbar(label="density of points")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.subplot(1,2,2)
plt.hist2d(dados_exemplo_2.ra, dados_exemplo_2.dec, bins=50)
plt.xlabel("R.A. (deg)", fontsize=14)
plt.ylabel("Dec. (deg)", fontsize=14)
plt.colorbar(label="density of points")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

*** 

Para o gráfico do CMD, vamos utilizar apenas o exemplo 2. 

In [ ]:
dados = dados_exemplo_2
del dados_exemplo_1   # limpando da memória os dataframes
del dados_exemplo_2   # que não vamos mais utilizar 

Por conveniência, podemos alterar os nomes de algumas colunas. 

In [ ]:
new_columns = {"coadd_object_id": "object_id",
               "mag_auto_g_dered": "mag_g",
               "mag_auto_r_dered": "mag_r",
               "mag_auto_i_dered": "mag_i",
               "magerr_auto_g": "err_g", 
               "magerr_auto_r": "err_r", 
               "magerr_auto_i": "err_i"}

dados.rename(columns=new_columns, inplace=True)

In [ ]:
dados.info()

In [ ]:
dados.head()

Cálculo da cor _g-r_ (nova coluna no _dataframe_ **dados**). 

In [ ]:
dados["gmr"] = dados.mag_g - dados.mag_r

In [ ]:
dados.head()

Limpeza da amostra: vamos selecionar apenas estrelas com fotometria de boa qualidade na banda _g_ (`flags_g < 4`) e com medidas bem sucedidas nas magnitudes ($mag \neq 99.$) . 

In [ ]:
dados.query("flags_g < 4  & mag_g != 99. & mag_r != 99. & mag_i != 99. ", inplace=True)

In [ ]:
dados.count()

In [ ]:
dados.describe()

#### CMD de Estrelas no DES DR2

In [ ]:
plt.figure(figsize=[7,5])
plt.title("CMD - Stars from DES DR2", fontsize=14)
plt.hexbin(dados.gmr, dados.mag_r, gridsize=500, bins='log')
plt.xlabel("$g - r$", fontsize=14)
plt.ylabel("$g$", fontsize=14)
plt.colorbar(label="density of points")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(-1.5,2.5)
plt.ylim(24,16)
plt.tight_layout()

## 3. LIneA User Query

<font color="red" > Em breve! </font>

--- 

Gostaria de sugerir alguma correção ou melhoria neste _notebook_? 

Fique à vontade para abrir um _issue_ no repositório [jupyterhub-tutorial](https://github.com/linea-it/jupyterhub-tutorial/issues) da organização [LIneA IT](https://github.com/linea-it/) no GitHub.  
